<h1 align=center><font size = 5>Compare the neighborhoods of the NYC and HKG and Business Recommendation</font></h1>

### The Battle of Neighborhoods (Week 1)

### Introduction
##### In this project,  we would leverage the Foursquare location data to explore and compare the neighborhoods of two multicultural city: New York City and Hong Kong City. We will determine how similar or dissimilar they are, visualize the neighborhoods in both City and their emerging clusters, and come up with some business ideas based on the segmented and clustered neighborhoods.

### Data section

##### For the NYC neighborhood data, we use the data provided by https://geo.nyu.edu
##### For the HKG neighborhood data, we use the data provided by www.latlong.net

##### NYC neighborhood data:  https://geo.nyu.edu/catalog/nyu_2451_34572
##### HKG neighborhood data: https://www.latlong.net/country/hong-kong-99.html

##### The NYC Neighborhood has a total of 5 boroughs and 306 neighborhoods while The HKG Neighborhood has a total of 1 boroughs and 25 neighborhoods . In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains all the boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

##### Once the data is in a structured format, we cluster the neighborhoods in the city of NYC and HKG.

### Week 1 end here

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests
from bs4 import BeautifulSoup
import re

print('Libraries imported.')

Libraries imported.


The following code accesses a file in my IBM Cloud Object Storage.

In [2]:
# The code was removed by Watson Studio for sharing.

,Borough,Neighborhood,Latitude,Longitude
0,Hong Kong,Sheung Wan,22.286394,114.149139
1,Hong Kong,Causeway Bay,22.285978,114.191490
2,Hong Kong,Kwai Chung,22.370157,114.123489
3,Hong Kong,Hong Kong Observatory,22.302219,114.174637
4,Hong Kong,Hong Kong International Airport,22.308046,113.918480


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(hongkong_data['Borough'].unique()),
        hongkong_data.shape[0]
    )
)

The dataframe has 1 boroughs and 25 neighborhoods.


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = 'FTRBKNH22E0XT30GLKUUHDNRQBP0S3XWABOJDK2T1TV5JJO3' # your Foursquare ID
CLIENT_SECRET = 'HGZVZK1UKIMJJ0DWEQPW2V1LOCI1W4K5LLTOLJRIGZNP3D4Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FTRBKNH22E0XT30GLKUUHDNRQBP0S3XWABOJDK2T1TV5JJO3
CLIENT_SECRET:HGZVZK1UKIMJJ0DWEQPW2V1LOCI1W4K5LLTOLJRIGZNP3D4Q


## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Hong Kong

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *hongkong_venues*.

In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

hongkong_venues = getNearbyVenues(names=hongkong_data['Neighborhood'],
                                   latitudes=hongkong_data['Latitude'],
                                   longitudes=hongkong_data['Longitude']
                                  )


Sheung Wan
Causeway Bay
Kwai Chung
Hong Kong Observatory
Hong Kong International Airport 
Tsing Yi
Quarry Bay
Shing Mun Country Park
Delta Hotel
Survitec - Tuen Mun
Rosedale Hotel Kowloon
Chuk Un
Central Plaza
Hong Kong Central Library
Bank of China Tower 
Cordis Hong Kong
International Finance Centre 
JW Marriott Hotel Hong Kong
International Commerce Centre
China Resources Building
Yiu Tung Public Library
Four Seasons Hotel Hong Kong
Victoria Harbour
Victoria Peak
Lan Kwai Fong


#### Let's check the size of the resulting dataframe

In [8]:
print(hongkong_venues.shape)

(1256, 7)


Let's check how many venues were returned for each neighborhood

In [9]:
hongkong_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bank of China Tower,77,77,77,77,77,77
Causeway Bay,89,89,89,89,89,89
Central Plaza,57,57,57,57,57,57
China Resources Building,42,42,42,42,42,42
Chuk Un,13,13,13,13,13,13
Cordis Hong Kong,89,89,89,89,89,89
Delta Hotel,100,100,100,100,100,100
Four Seasons Hotel Hong Kong,100,100,100,100,100,100
Hong Kong Central Library,77,77,77,77,77,77


#### Let's find out how many unique categories can be curated from all the returned venues

In [10]:
print('There are {} uniques categories.'.format(len(hongkong_venues['Venue Category'].unique())))

There are 188 uniques categories.


## 3. Analyze Each Neighborhood

In [11]:
# one hot encoding
hongkong_onehot = pd.get_dummies(hongkong_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hongkong_onehot['Neighborhood'] = hongkong_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hongkong_onehot.columns[-1]] + list(hongkong_onehot.columns[:-1])
hongkong_onehot = hongkong_onehot[fixed_columns]

hongkong_onehot.head()

,Neighborhood,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Donburi Restaurant,Dongbei Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Historic Site,History Museum,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Multiplex,Music Store,Nightclub,Non-Profit,Noodle House,Outdoor Supply Store,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Reservoir,Rest Area,Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Squash Court,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zhejiang Restaurant,Zoo
0,Sheung Wan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Sheung Wan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sheung Wan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Sheung Wan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [12]:
hongkong_onehot.shape

(1256, 189)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
hongkong_grouped = hongkong_onehot.groupby('Neighborhood').mean().reset_index()
hongkong_grouped

,Neighborhood,Accessories Store,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Donburi Restaurant,Dongbei Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Historic Site,History Museum,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Multiplex,Music Store,Nightclub,Non-Profit,Noodle House,Outdoor Supply Store,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Reservoir,Rest Area,Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Squash Court,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zhejiang Restaurant,Zoo
0,Bank of China Tower,0.00,0.00,0.0,0.00,0.0,0.000000,0.000000,0.012987,0.00,0.000000,0.000000,0.000000,0.012987,0.000000,0.012987,0.000000,0.000000,0.00,0.000000,0.012987,0.000000,0.012987,0.012987,0.012987,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.051948,0.000000,0.00,0.051948,0.000000,0.000000,0.025974,0.000000,0.000000,0.012987,0.051948,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.012987,0.012987,0.012987,0.000000,0.000000,0.025974,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038961,0.000000,0.012987,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.012987,0.012987,0.000000,0.000000,0.000000,0.000000,0.012987,0.00,0.064935,0.051948,0.000000,0.000000,0.000000,0.025974,0.000000,0.012987,0.000000,0.012987,0.000000,0.000000,0.000,0.051948,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.012987,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.038961,0.000000,0.000000,0.00,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.012987,0.025974,0.000000,0.00,0.00

#### Let's confirm the new size

In [14]:
hongkong_grouped.shape

(25, 189)

#### Let's print each neighborhood along with the top 5 most common venues

In [15]:
num_top_venues = 5

for hood in hongkong_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hongkong_grouped[hongkong_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bank of China Tower ----
                  venue  freq
0                 Hotel  0.06
1           Coffee Shop  0.05
2  Cantonese Restaurant  0.05
3                  Café  0.05
4             Hotel Bar  0.05


----Causeway Bay----
              venue  freq
0       Coffee Shop  0.08
1      Noodle House  0.06
2  Sushi Restaurant  0.06
3             Hotel  0.06
4              Café  0.04


----Central Plaza----
                  venue  freq
0  Hong Kong Restaurant  0.07
1  Cantonese Restaurant  0.05
2   Japanese Restaurant  0.05
3                 Hotel  0.05
4    Italian Restaurant  0.05


----China Resources Building----
                  venue  freq
0                 Hotel  0.07
1    Italian Restaurant  0.07
2                  Café  0.05
3  Hong Kong Restaurant  0.05
4  Cantonese Restaurant  0.05


----Chuk Un----
                  venue  freq
0  Fast Food Restaurant  0.15
1           Coffee Shop  0.15
2                  Park  0.08
3   Szechuan Restaurant  0.08
4                Temple  

#### Let's put that into a *pandas* dataframe

In [16]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hongkong_grouped['Neighborhood']

for ind in np.arange(hongkong_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hongkong_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bank of China Tower,Hotel,Cantonese Restaurant,Hotel Bar,Lounge,Café,Coffee Shop,French Restaurant,Park,Steakhouse,Dim Sum Restaurant
1,Causeway Bay,Coffee Shop,Noodle House,Hotel,Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Bakery,Cha Chaan Teng
2,Central Plaza,Hong Kong Restaurant,Japanese Restaurant,Cantonese Restaurant,Hotel,Coffee Shop,Chinese Restaurant,Italian Restaurant,Lounge,Spanish Restaurant,Café
3,China Resources Building,Hotel,Italian Restaurant,Japanese Restaurant,Snack Place,Chinese Restaurant,Cantonese Restaurant,Café,Hong Kong Restaurant,Szechuan Restaurant,Taiwanese Restaurant
4,Chuk Un,Coffee Shop,Fast Food Restaurant,Burger Joint,Szechuan Restaurant,Bus Stop,Park,Hong Kong Restaurant,Hakka Restaurant,Astrologer,Pool


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [18]:
# set number of clusters
kclusters = 5

hongkong_grouped_clustering = hongkong_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hongkong_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1,
       3, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hongkong_merged = hongkong_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hongkong_merged = hongkong_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

hongkong_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hong Kong,Sheung Wan,22.286394,114.149139,0,Japanese Restaurant,Coffee Shop,Café,French Restaurant,Hotel,Supermarket,Chinese Restaurant,Bar,Tapas Restaurant,Restaurant
1,Hong Kong,Causeway Bay,22.285978,114.191490,0,Coffee Shop,Noodle House,Hotel,Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Bakery,Cha Chaan Teng
2,Hong Kong,Kwai Chung,22.370157,114.123489,0,Dessert Shop,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Convenience Store,Shopping Mall,Coffee Shop,Café,Department Store,Gas Station
3,Hong Kong,Hong Kong Observatory,22.302219,114.174637,0,Japanese Restaurant,Cha Chaan Teng,Sushi Restaurant,Hotel,Noodle House,Dessert Shop,Steakhouse,Mexican Restaurant,Café,Snack Place
4,Hong Kong,Hong Kong International Airport,22.308046,113.918480,0,Airport Lounge,Airport Terminal,Coffee Shop,Accessories Store,Bar,Ramen Restaurant,Comic Shop,Electronics Store,Chinese Restaurant,Smoke Shop


let's visualize the resulting clusters

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined

#### Use geopy library to get the latitude and longitude values of Hong Kong.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [11]:
address = 'Hong Kong'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hong Kong are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hong Kong are 22.2793278, 114.1628131.


#### Scrape Popular Places in Hong Kong in the url and  the transform into a *pandas* dataframe

In [150]:
url = 'https://www.latlong.net/country/hong-kong-99.html'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'lxml')
li = []
for tag in soup.find_all("li"):
    li.append(tag.text)

In [151]:
#remove the non places item
li = li[25:52]
newli=[]
a= 0
for i in li:   
    newli.append(re.split(r'\(|\)|,',li[a]))
    a+=1

In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

NameError: name 'neighborhoods' is not defined